# Working with Databases 

## Inserting and Extracting Relational Data

In [ ]:
-- create a new table
CREATE TABLE users (
    id INT,
    name VARCHAR(256), 
    street VARCHAR(256),
    city VARCHAR(256),
    zip VARCHAR(9)
);

In [11]:
import psycopg2

# set connection string 
conn_string = "dbname='de-with-python' host='localhost'"

# creat a connection 
conn = psycopg2.connect(conn_string)

# create a cursor 
cur = conn.cursor()

In [14]:
# define a query 
query = """
insert into users (id,name,street,city,zip)
    values(%s,%s,%s,%s,%s)
"""

# supply data
data = (1,'Big Bird','Sesame Street','Fakeville','12345')

# preview what will be sent to the database 
cur.execute(query, data)

# commit to db 
conn.commit()

In [16]:
from faker import Faker 

# instantiate a faker, list to hold data, and id object
faker = Faker()
data = []
i = 2

# fill in and udpate values 
for r in range(1000):
    data.append((i, faker.name(), faker.street_address(), faker.city(), faker.zipcode()))
    i += 1

# convert to tuple for db upsertion 
db_tuple = tuple(data)

# execute the query for the items in the tuple
cur.executemany(query, db_tuple)

# commit the transaction 
conn.commit()